***GENERATED CODE FOR fraudclasstest PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Month', 'transformation_label': 'String Indexer'}], 'feature': 'Month', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Apr', 'max': 'Sep', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Month'}, {'feature_label': 'Month', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Month')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DayOfWeek', 'transformation_label': 'String Indexer'}], 'feature': 'DayOfWeek', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Friday', 'max': 'Wednesday', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DayOfWeek'}, {'feature_label': 'DayOfWeek', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DayOfWeek')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Make', 'transformation_label': 'String Indexer'}], 'feature': 'Make', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Accura', 'max': 'VW', 'missing': '0', 'distinct': '14'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Make'}, {'feature_label': 'Make', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Make')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AccidentArea', 'transformation_label': 'String Indexer'}], 'feature': 'AccidentArea', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Rural', 'max': 'Urban', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AccidentArea'}, {'feature_label': 'AccidentArea', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AccidentArea')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DayOfWeekClaimed', 'transformation_label': 'String Indexer'}], 'feature': 'DayOfWeekClaimed', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Friday', 'max': 'Wednesday', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DayOfWeekClaimed'}, {'feature_label': 'DayOfWeekClaimed', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DayOfWeekClaimed')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MonthClaimed', 'transformation_label': 'String Indexer'}], 'feature': 'MonthClaimed', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Apr', 'max': 'Sep', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MonthClaimed'}, {'feature_label': 'MonthClaimed', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MonthClaimed')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sex', 'transformation_label': 'String Indexer'}], 'feature': 'Sex', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sex'}, {'feature_label': 'Sex', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sex')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MaritalStatus', 'transformation_label': 'String Indexer'}], 'feature': 'MaritalStatus', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Married', 'max': 'Widow', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MaritalStatus'}, {'feature_label': 'MaritalStatus', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MaritalStatus')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Fault', 'transformation_label': 'String Indexer'}], 'feature': 'Fault', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Policy Holder', 'max': 'Third Party', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Fault'}, {'feature_label': 'Fault', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Fault')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PolicyType', 'transformation_label': 'String Indexer'}], 'feature': 'PolicyType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Sedan - All Perils', 'max': 'Utility - Liability', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PolicyType'}, {'feature_label': 'PolicyType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PolicyType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'VehicleCategory', 'transformation_label': 'String Indexer'}], 'feature': 'VehicleCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Sedan', 'max': 'Utility', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'VehicleCategory'}, {'feature_label': 'VehicleCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('VehicleCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'VehiclePrice', 'transformation_label': 'String Indexer'}], 'feature': 'VehiclePrice', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '20000 to 29000', 'max': 'more than 69000', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'VehiclePrice'}, {'feature_label': 'VehiclePrice', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('VehiclePrice')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Days_Policy_Accident', 'transformation_label': 'String Indexer'}], 'feature': 'Days_Policy_Accident', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1 to 7', 'max': 'none', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Days_Policy_Accident'}, {'feature_label': 'Days_Policy_Accident', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Days_Policy_Accident')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Days_Policy_Claim', 'transformation_label': 'String Indexer'}], 'feature': 'Days_Policy_Claim', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'more than 30', 'max': 'more than 30', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Days_Policy_Claim'}, {'feature_label': 'Days_Policy_Claim', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Days_Policy_Claim')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PastNumberOfClaims', 'transformation_label': 'String Indexer'}], 'feature': 'PastNumberOfClaims', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '1.0', 'stddev': '0.0', 'min': '1', 'max': 'none', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PastNumberOfClaims'}, {'feature_label': 'PastNumberOfClaims', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PastNumberOfClaims')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AgeOfVehicle', 'transformation_label': 'String Indexer'}], 'feature': 'AgeOfVehicle', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '2 years', 'max': 'new', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AgeOfVehicle'}, {'feature_label': 'AgeOfVehicle', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AgeOfVehicle')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AgeOfPolicyHolder', 'transformation_label': 'String Indexer'}], 'feature': 'AgeOfPolicyHolder', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '16 to 17', 'max': 'over 65', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AgeOfPolicyHolder'}, {'feature_label': 'AgeOfPolicyHolder', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AgeOfPolicyHolder')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PoliceReportFiled', 'transformation_label': 'String Indexer'}], 'feature': 'PoliceReportFiled', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PoliceReportFiled'}, {'feature_label': 'PoliceReportFiled', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PoliceReportFiled')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'WitnessPresent', 'transformation_label': 'String Indexer'}], 'feature': 'WitnessPresent', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'WitnessPresent'}, {'feature_label': 'WitnessPresent', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('WitnessPresent')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AgentType', 'transformation_label': 'String Indexer'}], 'feature': 'AgentType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'External', 'max': 'Internal', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AgentType'}, {'feature_label': 'AgentType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AgentType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'NumberOfSuppliments', 'transformation_label': 'String Indexer'}], 'feature': 'NumberOfSuppliments', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1 to 2', 'max': 'none', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'NumberOfSuppliments'}, {'feature_label': 'NumberOfSuppliments', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('NumberOfSuppliments')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AddressChange_Claim', 'transformation_label': 'String Indexer'}], 'feature': 'AddressChange_Claim', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1 year', 'max': 'under 6 months', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AddressChange_Claim'}, {'feature_label': 'AddressChange_Claim', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AddressChange_Claim')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'NumberOfCars', 'transformation_label': 'String Indexer'}], 'feature': 'NumberOfCars', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1 vehicle', 'max': 'more than 8', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'NumberOfCars'}, {'feature_label': 'NumberOfCars', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('NumberOfCars')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BasePolicy', 'transformation_label': 'String Indexer'}], 'feature': 'BasePolicy', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'All Perils', 'max': 'Liability', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BasePolicy'}, {'feature_label': 'BasePolicy', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BasePolicy')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run fraudclasstestHooks.ipynb
try:
	#sourcePreExecutionHook()

	fraudclaim = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Fraud Claim.csv', 'filename': 'Fraud Claim.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(fraudclaim)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run fraudclasstestHooks.ipynb
try:
	#transformationPreExecutionHook()

	fraudclasstestautofe = TransformationMain.run(fraudclaim,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Month", "transformation_label": "String Indexer"}], "feature": "Month", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Apr", "max": "Sep", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "WeekOfMonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.82", "stddev": "1.34", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "WeekOfMonth"}, {"transformationsData": [{"feature_label": "DayOfWeek", "transformation_label": "String Indexer"}], "feature": "DayOfWeek", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Friday", "max": "Wednesday", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DayOfWeek"}, {"transformationsData": [{"feature_label": "Make", "transformation_label": "String Indexer"}], "feature": "Make", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Accura", "max": "VW", "missing": "0", "distinct": "14"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Make"}, {"transformationsData": [{"feature_label": "AccidentArea", "transformation_label": "String Indexer"}], "feature": "AccidentArea", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Rural", "max": "Urban", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AccidentArea"}, {"transformationsData": [{"feature_label": "DayOfWeekClaimed", "transformation_label": "String Indexer"}], "feature": "DayOfWeekClaimed", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Friday", "max": "Wednesday", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DayOfWeekClaimed"}, {"transformationsData": [{"feature_label": "MonthClaimed", "transformation_label": "String Indexer"}], "feature": "MonthClaimed", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Apr", "max": "Sep", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MonthClaimed"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "WeekOfMonthClaimed", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.6", "stddev": "1.28", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "WeekOfMonthClaimed"}, {"transformationsData": [{"feature_label": "Sex", "transformation_label": "String Indexer"}], "feature": "Sex", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sex"}, {"transformationsData": [{"feature_label": "MaritalStatus", "transformation_label": "String Indexer"}], "feature": "MaritalStatus", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Married", "max": "Widow", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MaritalStatus"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "37.25", "stddev": "15.38", "min": "0", "max": "80", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Fault", "transformation_label": "String Indexer"}], "feature": "Fault", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Policy Holder", "max": "Third Party", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Fault"}, {"transformationsData": [{"feature_label": "PolicyType", "transformation_label": "String Indexer"}], "feature": "PolicyType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Sedan - All Perils", "max": "Utility - Liability", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PolicyType"}, {"transformationsData": [{"feature_label": "VehicleCategory", "transformation_label": "String Indexer"}], "feature": "VehicleCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Sedan", "max": "Utility", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "VehicleCategory"}, {"transformationsData": [{"feature_label": "VehiclePrice", "transformation_label": "String Indexer"}], "feature": "VehiclePrice", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "20000 to 29000", "max": "more than 69000", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "VehiclePrice"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FraudFound_P", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.06", "stddev": "0.24", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FraudFound_P"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PolicyNumber", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "250.5", "stddev": "144.48", "min": "1", "max": "500", "missing": "0"}, "updatedLabel": "PolicyNumber"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RepNumber", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "8.25", "stddev": "4.59", "min": "1", "max": "16", "missing": "0"}, "updatedLabel": "RepNumber"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Deductible", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "405.2", "stddev": "35.43", "min": "300", "max": "700", "missing": "0"}, "updatedLabel": "Deductible"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DriverRating", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.57", "stddev": "1.13", "min": "1", "max": "4", "missing": "0"}, "updatedLabel": "DriverRating"}, {"transformationsData": [{"feature_label": "Days_Policy_Accident", "transformation_label": "String Indexer"}], "feature": "Days_Policy_Accident", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1 to 7", "max": "none", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Days_Policy_Accident"}, {"transformationsData": [{"feature_label": "Days_Policy_Claim", "transformation_label": "String Indexer"}], "feature": "Days_Policy_Claim", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "more than 30", "max": "more than 30", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Days_Policy_Claim"}, {"transformationsData": [{"feature_label": "PastNumberOfClaims", "transformation_label": "String Indexer"}], "feature": "PastNumberOfClaims", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "none", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PastNumberOfClaims"}, {"transformationsData": [{"feature_label": "AgeOfVehicle", "transformation_label": "String Indexer"}], "feature": "AgeOfVehicle", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "2 years", "max": "new", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AgeOfVehicle"}, {"transformationsData": [{"feature_label": "AgeOfPolicyHolder", "transformation_label": "String Indexer"}], "feature": "AgeOfPolicyHolder", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "16 to 17", "max": "over 65", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AgeOfPolicyHolder"}, {"transformationsData": [{"feature_label": "PoliceReportFiled", "transformation_label": "String Indexer"}], "feature": "PoliceReportFiled", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PoliceReportFiled"}, {"transformationsData": [{"feature_label": "WitnessPresent", "transformation_label": "String Indexer"}], "feature": "WitnessPresent", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "WitnessPresent"}, {"transformationsData": [{"feature_label": "AgentType", "transformation_label": "String Indexer"}], "feature": "AgentType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "External", "max": "Internal", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AgentType"}, {"transformationsData": [{"feature_label": "NumberOfSuppliments", "transformation_label": "String Indexer"}], "feature": "NumberOfSuppliments", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1 to 2", "max": "none", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "NumberOfSuppliments"}, {"transformationsData": [{"feature_label": "AddressChange_Claim", "transformation_label": "String Indexer"}], "feature": "AddressChange_Claim", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1 year", "max": "under 6 months", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AddressChange_Claim"}, {"transformationsData": [{"feature_label": "NumberOfCars", "transformation_label": "String Indexer"}], "feature": "NumberOfCars", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1 vehicle", "max": "more than 8", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "NumberOfCars"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1994.0", "stddev": "0.0", "min": "1994", "max": "1994", "missing": "0"}, "updatedLabel": "Year"}, {"transformationsData": [{"feature_label": "BasePolicy", "transformation_label": "String Indexer"}], "feature": "BasePolicy", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "All Perils", "max": "Liability", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BasePolicy"}]}))

	#transformationPostExecutionHook(fraudclasstestautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run fraudclasstestHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(fraudclasstestautofe, ["WeekOfMonth", "WeekOfMonthClaimed", "Age", "PolicyNumber", "RepNumber", "Deductible", "DriverRating", "Year", "Month_stringindexer", "DayOfWeek_stringindexer", "Make_stringindexer", "AccidentArea_stringindexer", "DayOfWeekClaimed_stringindexer", "MonthClaimed_stringindexer", "Sex_stringindexer", "MaritalStatus_stringindexer", "Fault_stringindexer", "PolicyType_stringindexer", "VehicleCategory_stringindexer", "VehiclePrice_stringindexer", "Days_Policy_Accident_stringindexer", "Days_Policy_Claim_stringindexer", "PastNumberOfClaims_stringindexer", "AgeOfVehicle_stringindexer", "AgeOfPolicyHolder_stringindexer", "PoliceReportFiled_stringindexer", "WitnessPresent_stringindexer", "AgentType_stringindexer", "NumberOfSuppliments_stringindexer", "AddressChange_Claim_stringindexer", "NumberOfCars_stringindexer", "BasePolicy_stringindexer"], "FraudFound_P")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

